# Setup

In [41]:
#import ujson as json
import gzip
import pandas as pd
from collections import Counter
import string
from multiprocessing import Pool
import numpy as np
from collections import defaultdict
from bs4 import BeautifulSoup
import requests
import re
import langid
import pickle
import ujson as json
import sys
from glob import glob
import os
import pickle
# For field matching
import field_matching
from tqdm import tqdm
STOPWORDS = set(['and','not','of','the'])

%matplotlib inline
from scipy import stats, integrate
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

In [42]:
tr = dict((ord(x), None) for x in string.punctuation)

def lookup(myjson, k):
  # return myjson[k]
  if '.' in k:
    # jpath path
    ks = k.split('.')
    v = myjson
    for k in ks: 
        if not v:
            return ""
        v = v.get(k,{})
    return v or ""
  return myjson.get(k,"")


def get_terms(text, term_set =None):
    spl = re.split("[ /+,]",text.lower())
    t = set([x.translate(tr) for x in spl])
    if term_set:
        return t & term_set
    return t



def clean_dict(d):
    for k, v in d.items():
        if v:
            d[k] = v.text
    return d
        
def gen_bio(user_dat):
    person = user_dat.find("person:person")
    if not person:
        return {
             "person_created_prof_date" : None,
             "person_last_mod_date" : None,
             "person_first_name" : None,
             "person_last_name" : None
        }
    person = {
         "person_created_prof_date" : person.find('common:created-date'),
         "person_last_mod_date" : person.find('common:last-modified-date'),
         "person_first_name" : person.find('personal-details:given-names'),
         "person_last_name" : person.find('personal-details:family-name')
    }
    return clean_dict(person)

def gen_affl(user_dat,ty):
    affiliations = []
    for e in user_dat.find_all("{}:{}-summary".format(ty,ty)):
        affl_dict = {}

        org = e.find("common:organization".format(ty))
        if org:
            affl_dict['org_name'] = org.find("common:name")
            addr = org.find("common:address")
            if addr:
                affl_dict['org_country'] = addr.find('common:country')
            else:
                affl_dict['org_country'] = None
        else:
            affl_dict['org_name'] = None
            affl_dict['org_country'] = None

        startdate = e.find("common:start-date")
        enddate = e.find("common:end-date")
        if startdate:
            affl_dict['start_month'] = startdate.find("common:month")
            affl_dict['start_year'] = startdate.find("common:year")
        elif enddate:
            affl_dict['start_month'] = enddate.find("common:month")
            affl_dict['start_year'] = enddate.find("common:year")
        else:
            affl_dict['start_month'] = None
            affl_dict['start_year'] = None
        
        affl_dict['role'] = e.find("common:role-title".format(ty))
        affl_dict['dept'] = e.find("common:department-name".format(ty))

        affl_dict = clean_dict(affl_dict)
        
        affl_dict['type'] = ty
        
        disamb_org = e.find("common:disambiguated-organization")
        if disamb_org:
            affl_dict['disamb_org'] = disamb_org.text.strip().replace("\n", " ")
        else:
            affl_dict['disamb_org'] = None
        affiliations.append(affl_dict)
    return affiliations

def gen_orcid(user_dat):
    orcid = user_dat.find("common:orcid-identifier")
    if orcid:
        oi = orcid.find("common:path")
        if oi:
            return oi.text
    return None


# Load in Bio, Affiliation data

# Code for raw dump


In [43]:
DATA_DIR = "/net/twitter/kenny-data/branching_pipeline/"
BIOS_DIR = os.path.join(DATA_DIR, "bios_dir")
AFFILIATIONS_DIR = os.path.join(DATA_DIR, "affiliations_dir")

In [44]:
import os
if os.path.exists(BIOS_DIR):
    print('SKIP THIS SECTION YOU HAVE THE PREPROCESSED DATA GO TO THE SECTION STARTING WITH Code for Pre-processed...')

SKIP THIS SECTION YOU HAVE THE PREPROCESSED DATA GO TO THE SECTION STARTING WITH Code for Pre-processed...


In [26]:
try:
    os.mkdir(BIOS_DIR)
    os.mkdir(AFFILIATIONS_DIR)
except:
    print("Dirs already created, you probably have the data")

In [28]:
all_summaries_toplevel = glob(os.path.join(DATA_DIR,"ORCID_2020_10_summaries","*"))


'000'

In [ ]:

def gen_fil_from_toplevel(toplevel_dir):
    bios = []
    affiliations = []
    for i,fil in enumerate(glob(os.path.join(toplevel_dir,"*"))):
        user_dat =BeautifulSoup(open(fil))
        userinfo = {"orcid": gen_orcid(user_dat),
                     "bio" : gen_bio(user_dat),
                     "affiliations":  gen_affl(user_dat, "employment") + gen_affl(user_dat, "education")
                   }
        oid = userinfo['orcid']
        for af in userinfo['affiliations']:
            af['oid'] = oid
            affiliations.append(af)
        userinfo['bio']['oid'] = oid
        bios.append(userinfo['bio'])
        
    basename_dir = os.path.basename(toplevel_dir)
    with gzip.open(os.path.join(DATA_DIR,"bios_dir",
                                basename_dir+".json.gz"),
                   "w") as bio_out:
        for bio in bios:
            bio_out.write((json.dumps(bio)+"\n").encode())
    with gzip.open(os.path.join(DATA_DIR,"affiliations_dir",
                                basename_dir+".json.gz"),
                    "w") as affiliations_out:
        for affiliation in affiliations:
            affiliations_out.write((json.dumps(affiliation)+"\n").encode())
    print(toplevel_dir)
    

from multiprocessing import Pool

with Pool(processes=6) as pool:
    pool.map(gen_fil_from_toplevel, all_summaries_toplevel)


        


/net/twitter/kenny-data/branching_pipeline/ORCID_2020_10_summaries/168
/net/twitter/kenny-data/branching_pipeline/ORCID_2020_10_summaries/042
/net/twitter/kenny-data/branching_pipeline/ORCID_2020_10_summaries/000
/net/twitter/kenny-data/branching_pipeline/ORCID_2020_10_summaries/084
/net/twitter/kenny-data/branching_pipeline/ORCID_2020_10_summaries/126
/net/twitter/kenny-data/branching_pipeline/ORCID_2020_10_summaries/20X
/net/twitter/kenny-data/branching_pipeline/ORCID_2020_10_summaries/001
/net/twitter/kenny-data/branching_pipeline/ORCID_2020_10_summaries/169
/net/twitter/kenny-data/branching_pipeline/ORCID_2020_10_summaries/043
/net/twitter/kenny-data/branching_pipeline/ORCID_2020_10_summaries/085
/net/twitter/kenny-data/branching_pipeline/ORCID_2020_10_summaries/127
/net/twitter/kenny-data/branching_pipeline/ORCID_2020_10_summaries/210
/net/twitter/kenny-data/branching_pipeline/ORCID_2020_10_summaries/002
/net/twitter/kenny-data/branching_pipeline/ORCID_2020_10_summaries/16X
/net/t

# Code for pre-processed dumps

In [47]:
bios = []
for fil in glob(os.path.join(BIOS_DIR,"*")):
    with gzip.open(fil) as inf:
        bios += [json.loads(b) for b in inf]

affiliations = []
for fil in glob(os.path.join(AFFILIATIONS_DIR,"*")):
    with gzip.open(fil) as inf:
        affiliations += [json.loads(a) for a in inf]

# Data Frame for bios/affiliations


In [48]:
# # Create bio dataframe
bio_df = pd.DataFrame(bios)

In [49]:
bio_df.shape

(9747030, 5)

In [50]:
# Create the DF 
df = pd.DataFrame(affiliations)

# merge affiliations with bios
df = pd.merge(df,bio_df, on="oid")

In [51]:
bio_df.shape, bio_df.oid.nunique()

((9747030, 5), 9600248)

In [52]:
bio_df = bio_df.groupby("oid").last().reset_index()

In [53]:
bio_df.head()

,oid,person_created_prof_date,person_last_mod_date,person_first_name,person_last_name
0,0000-0001-5000-0015,2016-04-15T20:44:30.144Z,2016-04-15T20:44:30.144Z,Shittu,Olufunke
1,0000-0001-5000-0023,2019-06-10T10:59:37.348Z,2019-06-10T11:04:55.174Z,Alisher,Ibrayev
2,0000-0001-5000-0031,2018-03-14T00:58:04.681Z,2018-03-14T00:58:04.908Z,Jieun,Kim
3,0000-0001-5000-0066,2018-11-28T19:03:37.722Z,2018-11-28T19:03:37.950Z,Ivan,Salazar
4,0000-0001-5000-0074,2017-03-31T02:53:39.703Z,2018-09-27T02:18:44.516Z,Susan,Luong


In [54]:
df.head()

,org_name,org_country,start_month,start_year,role,dept,type,disamb_org,oid,person_created_prof_date,person_last_mod_date,person_first_name,person_last_name
0,Niagara College,CA,09,1976,Library Technician Diploma,None,education,125147 RINGGOLD,0000-0001-5109-1000,2019-05-01T13:04:57.507Z,2019-05-01T13:45:47.727Z,Andrew,Porteus
1,"PCSIR Labs. Complex, Ferozepur Road",PK,03,2001,Scientific Officer,Applied Chemistry Research Centre,employment,None,0000-0001-5042-7000,2016-04-15T20:46:17.804Z,2016-04-15T20:46:17.804Z,Muhammad Usman,Sabri
2,University of the Punjab,PK,None,2001,Master,Chemistry,education,66906 RINGGOLD,0000-0001-5042-7000,2016-04-15T20:46:17.804Z,2016-04-15T20:46:17.804Z,Muhammad Usman,Sabri
3,University College London Hospitals NHS Founda...,GB,07,2014,Consultant Neuroradiologist,Lysholm Department of Neuroradiology,employment,98546 RINGGOLD,0000-0001-5136-6000,2018-02-07T09:59:17.219Z,2018-02-07T09:59:17.444Z,Steffi,Thust
4,University of Al-Ameed,IQ,None,None,Dr.,Pharmacology,employment,None,0000-0001-5002-1000,2019-03-19T10:17:07.129Z,2019-03-19T10:17:07.366Z,Sameer,Abbood


In [55]:
df.shape

(8146175, 13)

# Affiliations per person

We probably want to toss people who have a crazy number of affiliations

In [56]:
# How many affiliations per person
affiliations_per_person = df.groupby("oid").size()

In [57]:
affiliations_per_person.shape

(3006777,)

In [58]:
affl_percentage_cum = (affiliations_per_person.value_counts()/float(len(affiliations_per_person))).cumsum()

In [59]:
affiliations_per_person[affiliations_per_person > 1].shape

(1939081,)

In [60]:
# Cumsum of affiliations per person - we can toss ppl with > 12 affiliations and lose only .05% of the data
affl_percentage_cum.head(15)

#what are these 2 affiliations?

1     0.355097
2     0.635458
3     0.766007
4     0.855954
5     0.909426
6     0.942817
7     0.963675
8     0.976755
9     0.984806
10    0.989952
11    0.993112
12    0.995240
13    0.996622
14    0.997552
15    0.998184
dtype: float64

In [61]:
affiliations_per_person[ affiliations_per_person > 15][:10]

oid
0000-0001-5009-0233    19
0000-0001-5026-5865    16
0000-0001-5026-7115    16
0000-0001-5027-9837    22
0000-0001-5033-9086    23
0000-0001-5034-8476    18
0000-0001-5043-5174    16
0000-0001-5043-7719    17
0000-0001-5048-8495    20
0000-0001-5050-0810    17
dtype: int64

In [62]:
# Remove ppl with > 12 affiliations

df = df[df.oid.isin(affiliations_per_person[affiliations_per_person < 13].index.values)]
print(df.shape)
# How many ppl. do we have a this point?
df.oid.nunique()

(7726874, 13)


2992464

# Role Categorization

Using a series of regular expressions, we categorize what role a given affiliation represents.  
We determine a role by the highest matching role we identify (using the regular expressions, 
in the order of roles in the code below).  If there is no match, we will give the affiliation
a blank role

## Clean Role text

In [63]:
clean_role = df.role.apply(lambda x : x.strip().replace(".","").lower() if x else None)
df = df.assign(clean_role=clean_role)
roles = df.clean_role.value_counts()

In [64]:
df

,org_name,org_country,start_month,start_year,role,dept,type,disamb_org,oid,person_created_prof_date,person_last_mod_date,person_first_name,person_last_name,clean_role
0,Niagara College,CA,09,1976,Library Technician Diploma,None,education,125147 RINGGOLD,0000-0001-5109-1000,2019-05-01T13:04:57.507Z,2019-05-01T13:45:47.727Z,Andrew,Porteus,library technician diploma
1,"PCSIR Labs. Complex, Ferozepur Road",PK,03,2001,Scientific Officer,Applied Chemistry Research Centre,employment,None,0000-0001-5042-7000,2016-04-15T20:46:17.804Z,2016-04-15T20:46:17.804Z,Muhammad Usman,Sabri,scientific officer
2,University of the Punjab,PK,None,2001,Master,Chemistry,education,66906 RINGGOLD,0000-0001-5042-7000,2016-04-15T20:46:17.804Z,2016-04-15T20:46:17.804Z,Muhammad Usman,Sabri,master
3,University College London Hospitals NHS Founda...,GB,07,2014,Consultant Neuroradiologist,Lysholm Department of Neuroradiology,employment,98546 RINGGOLD,0000-0001-5136-6000,2018-02-07T09:59:17.219Z,2018-02-07T09:59:17.444Z,Steffi,Thust,consultant neuroradiologist
4,University of Al-Ameed,IQ,None,None,Dr.,Pharmacology,employment,None,0000-0001-5002-1000,2019-03-19T10:17:07.129Z,2019-03-19T10:17:07.366Z,Sameer,Abbood,dr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8146170,Macquarie University,AU,None,None,None,None,employment,7788 RINGGOLD,0000-0003-4957-3961,2016-09-08T00:54:38.504Z,2020-05-14T21:23:32.385Z,Jonathan,Williamson,None
8146171,Macquarie University Hospital,AU,01,2012,Visiting Medical Officer,Macquarie Respiratory and Sleep,employment,150782 RINGGOLD,0000-0003-4957-3961,2016-09-08T00:54:38.504Z,2020-05-14T21:23:32.385Z,Jonathan,Williamson,visiting medical officer
8146172,South Western Sydney Local Health District,AU,02,2010,Staff Specialist,Respiratory and Sleep Medicine,employment,1511 RINGGOLD,0000-0003-4957-3961,2016-09-08T00:54:38.504Z,2020-05-14T21:23:32.385Z,Jonathan,Williamson,staff specialist
8146173,University of Western Australia,AU,02,2007,Phd,Anatomy and Biological Sciences,education,2720 RINGGOLD,0000-0003-4957-3961,2016-09-08T00:54:38.504Z,2020-05-14T21:23:32.385Z,Jonathan,Williamson,phd


## Find roles

In [65]:
regexes = [
 ["research", re.compile("research|scientist|scholar|ricercatore")],
 ["bachelors", re.compile("(\b|^|\s)((b[ \.]?(a|s|e|se|sc|s\.e|sn)(\b|$|\s|\.))|bachelor|btech|underg|licenciatura|graduação)")],
 ["masters/postgrad", re.compile("(\b|^|\s)((m[\. ]?(a|s|p|sc|as)(\b|$|\s|\.))|master|^me$|mlis|mba|mbbs|postgrad|m[ \.]?tech|mphil|mph|mestrado|magister|mbchb|meng|mlis)")],
 ["phd",re.compile("^doctor(ate)?$|^d[\.]?r[\.]?$|^m[\.]?d[\.]?$|ph[ \.]?d(\b|$|\s|\.)|doctor of|d(\. )?phil|doctorado|mestre|graduate student|(\b|\s|^)graduate (research|teaching)? ?assistant|pharmd|^jd$|doctoral student|doutor|doctorat")],
 ["postdoc", re.compile("post[ o-]?d")],
 ["prof", re.compile(u"prof|(\b|^|\s)lect|faculty|reader|docente|instructor|доцент")],
 ['head', re.compile("dean|director|head")]
]

# for x in ['msc','m sc','manager','ma','ba ', 'bm', 'mas','b.a.', 'bastard', 'b.s.e.','d.r.','phd','phda'] +roles[100:200].index.tolist() :
#     print x
#     for name, reg in regexes:
#         if reg.search(x):
#             print "\t", name


In [66]:
matches = defaultdict(list)
captured = 0
total = 0
for i, x in roles.reset_index().iterrows():
    matched = ""
    for name, reg in regexes:
        if reg.search(x[0]):
            matched = name
    if matched != "":
        captured += x[1]
    total += x[1]
    matches[matched].append(x[0])

In [67]:
print(captured, total )
print('Percentage of non-null affiliations we can identify a role for: ', float(captured)/total)

4756244 6770376
Percentage of non-null affiliations we can identify a role for:  0.7025081029473105


In [68]:
print(len(matches['']), len(roles))
print('Percent of all unique role fields we can identify: ', len(matches[''])/float(len(roles)))

625590 1344173
Percent of all unique role fields we can identify:  0.4654088424629865


In [69]:
role_df=pd.DataFrame([x for k,v in matches.items() for x in zip(v,[k]*len(v))],
                     columns=['clean_role','role_category'])

In [70]:
df = df.merge(role_df,on="clean_role",how = "left")

## Check out role details

In [71]:
missing = df[pd.isnull(df.role_category)].shape[0]+df[df.role_category == ""].shape[0]
print(missing, float(missing)/len(df))
df.role_category.value_counts()

2970630 0.3844543084305503


                    2014132
phd                 1272007
prof                1190897
masters/postgrad     803589
bachelors            592812
research             484244
postdoc              253086
head                 159609
Name: role_category, dtype: int64

In [72]:
print('How many people have at least 1 category? ',)
print(df[~((pd.isnull(df.role_category)) | (df.role_category == ""))].oid.nunique())

How many people have at least 1 category? 
2152229


In [73]:
# the type field isn't named well, rename
df.rename(index=str, columns={"type" : "emp_or_edu"}, inplace=True)

print(df.emp_or_edu.value_counts())

# BUT! If the type is "employment", then we shouldn't be assigning anything but prof, head, research or postdoc?
# Actually these look fine, lets leave it
df.loc[(df.emp_or_edu == "employment") & (df.role_category.isin(["phd"]))].clean_role.value_counts()

education     3952556
employment    3774318
Name: emp_or_edu, dtype: int64


phd student                                 44336
dr                                          13551
phd                                         12799
phd candidate                               12748
graduate research assistant                  8319
                                            ...  
bolseira de investigação - mestre               1
ph d student and assistant                      1
phd by published work co-supervision            1
ph d of economic sciences                       1
teaching and research graduate assistant        1
Name: clean_role, Length: 14517, dtype: int64

# Translation of Department fields

## If you haven't done the translation yet you have to run all of this

In [74]:
import os
if os.path.exists("translated_fieldnames_2020.csv"):
    print('SKIP THIS SECTION YOU HAVE THE TRANSLATED DATA GO TO THE SECTION STARTING WITH "Otherwise"...')

In [168]:
#NSF Fields

tab = requests.get("https://www.nsf.gov/statistics/2017/nsf17306/datatables/tab-13.htm")
s = BeautifulSoup(tab.text)
tab = s.find("table",{"id" : "data-table"})

def get_text_indent(row):
    text = row.text.replace(", general","").replace(", other","").lower().strip()
    indent = int(row['class'][0].replace("indent_",""))
    return text, indent

nsf_field_hierarchy = defaultdict(list)
nsf_terms = set()
rows = tab.find_all("th", {"scope" : "row"})

# stupid but it works
i = 1
while i < len(rows):
    # get my info
    curr_text, curr_indent  = get_text_indent(rows[i])
    # make sure this isn't already in the tree
    if curr_text not in nsf_terms:
        nsf_terms.add(curr_text)
        for j in reversed(range(i)):
            prev_text, prev_indent = get_text_indent(rows[j])
            if curr_indent - prev_indent == 1:
                nsf_field_hierarchy[prev_text].append(curr_text)
                break
    #else:
        #print rows[i].text, curr_text, ' already in tree'
    i += 1

nsf_termlist = Counter()
for x in nsf_terms:
    nsf_termlist.update([x for x in get_terms(x) if x not in STOPWORDS])
nsf_termlist = set(nsf_termlist.keys())

In [170]:
# Wikipedia Fields
#https://en.wikipedia.org/wiki/Outline_of_academic_disciplines
def get_text_indent_wiki(row):
    indent = -1
    if row.startswith("==="):
        indent = 1
    elif row.startswith("=="):
        indent = 0
    elif row.startswith("'''"):
        indent = 2
    elif row.startswith("***"):
        indent = 5
    elif row.startswith("**"):
        indent = 4
    elif row.startswith("*"):
        indent = 3
    else:
        pass
        #print "NOPE", row
    text = row.replace("*","").replace("=","").lower().strip()
    
    text = text.replace("'''","")

    text = re.sub("(#|\|)[a-z A-Z\(\)]+\]\]","",text)
    text = text.replace("[[","").replace("]]","").strip()
    if text.rfind("(outline") != -1:
        text = text[:text.rfind("(outline")]
    return text.strip(), indent

wiki_field_hierarchy = defaultdict(list)
rows = [get_text_indent_wiki(x) for x in open("wiki_page.txt").readlines()]
rows = [x for x in rows if x[1] != -1]
wiki_terms = set()

# stupid but it works
i = 1
while i < len(rows):
    # get my info
    curr_text, curr_indent  = rows[i]
    # make sure this isn't already in the tree
    if curr_text not in wiki_terms:
        wiki_terms.add(curr_text)
        for j in reversed(range(i)):
            prev_text, prev_indent = rows[j]
            if curr_indent - prev_indent > 0:
                wiki_field_hierarchy[prev_text].append(curr_text)
                break
    i += 1
    
wiki_termlist = Counter()
for x in wiki_terms:
    wiki_termlist.update([x for x in get_terms(x) if x not in STOPWORDS])
wiki_termlist = set(wiki_termlist.keys())


for term in wiki_terms:
    if term not in wiki_field_hierarchy:
        wiki_field_hierarchy[term] = []
        
        
# Archaeology is a social science discipline - https://en.wikipedia.org/wiki/Archaeology
# So is communications
wiki_field_hierarchy['social sciences'].append("archaelogy")
wiki_terms.add("archaeology")
wiki_field_hierarchy['social sciences'].append("communications")
wiki_terms.add("communications")

# if Music history is a subset of Music, art history should be a subset of art
wiki_field_hierarchy["visual arts"].append("art history")
wiki_terms.add("art history")

# Classics == classical literature
wiki_field_hierarchy["history of literature"].append("classics")
wiki_field_hierarchy["history of literature"].append("classical literature")
wiki_terms.add("classics")
wiki_terms.add("classical literature")

# If English lit is under language and arts, so is spanish
wiki_field_hierarchy["languages and literature"].append("spanish literature")
wiki_terms.add("spanish literature") 


In [171]:
# Bring back in cleaned version and count number of terms in each individual's department
manual_termlist = {x.split("\t")[0] for x in io.open("manual_keywords.tsv")}

In [172]:
# Write out the fields that we need to translate to English
non_eng_affl = df[~df.org_country.isin(["US","UK", "GB", "IE"])].dept.unique()
print(len(non_eng_affl))


1157104


In [ ]:
to_translate = set()
for i,x in enumerate(non_eng_affl):
    if i % 30000 == 0:
        print(i)
    if x:
        terms = get_terms(x)
        if (len(terms & nsf_termlist) or 
            len(terms & wiki_termlist) or 
            len(terms & manual_termlist) or 
            langid.classify(x)[0] == 'en'):
                continue
        to_translate.add(x.lower())

0
30000
60000
90000
120000
150000
180000
210000
240000
270000
300000
330000
360000
390000
420000
450000
480000
510000
540000
570000
600000


In [174]:
already_translated = pd.read_csv("translated_fieldnames_2020.csv",encoding="utf8")
#already_translated.columns = ['from','to']
print(len(set(to_translate)), len(set(already_translated['from'].values)), 
      len(set(to_translate) - set(already_translated['from'].values)))


224821 160156 73072


In [192]:
# trans = []
# l = 0
# with open("./translated_2020.tsv") as inf:
#     for i,line in enumerate(inf):
#         s = line.split("\t")
#         if len(s) > 2:
#             l += 1
#         else:
#             trans.append(s)
# trans = pd.DataFrame(trans)
# trans.columns = ['from_field','to_field']
# trans.head()

,from_field,to_field
0,sistema integrado de bibliotecas regionais,integrated regional libraries\n
1,bidang kewirausahaan,entrepreneurship\n
2,instituto de química; departamento de química ...,Chemistry Institute; of inorganic chemistry de...
3,mi̇mari̇ proje ve satiş,ARCHITECTURAL PROJECT and sale\n
4,grupo de trastornos endocrinos,endocrine disorders group\n


In [175]:
with open("./to_translate.csv","w") as of:
    for x in set(to_translate) - set(already_translated['from'].values):
         of.write(x + "\n")

# Otherwise, if you have the pre-translated data, just run this!

In [75]:
translated = pd.read_csv("translated_fieldnames_2020.csv",encoding="utf8",header=None)
translated.columns = ['from_field','to_field']

In [76]:
translated.drop_duplicates("from_field",inplace=True)
translated.from_field = translated.from_field.str.lower()
translated.to_field = translated.to_field.str.lower()


In [77]:
df.dept = df.dept.str.lower()
df = pd.merge(df, translated,left_on="dept",right_on="from_field",how="left")
df.dept = df.apply(lambda x : x['dept'] if pd.isnull(x['to_field']) else x['to_field'],axis=1)
df.drop(['from_field','to_field'],axis=1,inplace=True)
df.shape

(7726874, 15)

# Field Matching

## Clean department names

In [78]:
df[~pd.isnull(df.org_name)]

,org_name,org_country,start_month,start_year,role,dept,emp_or_edu,disamb_org,oid,person_created_prof_date,person_last_mod_date,person_first_name,person_last_name,clean_role,role_category
0,Niagara College,CA,09,1976,Library Technician Diploma,None,education,125147 RINGGOLD,0000-0001-5109-1000,2019-05-01T13:04:57.507Z,2019-05-01T13:45:47.727Z,Andrew,Porteus,library technician diploma,
1,"PCSIR Labs. Complex, Ferozepur Road",PK,03,2001,Scientific Officer,applied chemistry research centre,employment,None,0000-0001-5042-7000,2016-04-15T20:46:17.804Z,2016-04-15T20:46:17.804Z,Muhammad Usman,Sabri,scientific officer,
2,University of the Punjab,PK,None,2001,Master,chemistry,education,66906 RINGGOLD,0000-0001-5042-7000,2016-04-15T20:46:17.804Z,2016-04-15T20:46:17.804Z,Muhammad Usman,Sabri,master,masters/postgrad
3,University College London Hospitals NHS Founda...,GB,07,2014,Consultant Neuroradiologist,lysholm department of neuroradiology,employment,98546 RINGGOLD,0000-0001-5136-6000,2018-02-07T09:59:17.219Z,2018-02-07T09:59:17.444Z,Steffi,Thust,consultant neuroradiologist,
4,University of Al-Ameed,IQ,None,None,Dr.,pharmacology,employment,None,0000-0001-5002-1000,2019-03-19T10:17:07.129Z,2019-03-19T10:17:07.366Z,Sameer,Abbood,dr,phd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7726869,Macquarie University,AU,None,None,None,None,employment,7788 RINGGOLD,0000-0003-4957-3961,2016-09-08T00:54:38.504Z,2020-05-14T21:23:32.385Z,Jonathan,Williamson,None,NaN
7726870,Macquarie University Hospital,AU,01,2012,Visiting Medical Officer,macquarie respiratory and sleep,employment,150782 RINGGOLD,0000-0003-4957-3961,2016-09-08T00:54:38.504Z,2020-05-14T21:23:32.385Z,Jonathan,Williamson,visiting medical officer,
7726871,South Western Sydney Local Health District,AU,02,2010,Staff Specialist,respiratory and sleep medicine,employment,1511 RINGGOLD,0000-0003-4957-3961,2016-09-08T00:54:38.504Z,2020-05-14T21:23:32.385Z,Jonathan,Williamson,staff specialist,
7726872,University of Western Australia,AU,02,2007,Phd,anatomy and biological sciences,education,2720 RINGGOLD,0000-0003-4957-3961,2016-09-08T00:54:38.504Z,2020-05-14T21:23:32.385Z,Jonathan,Williamson,phd,phd


In [79]:
df = df.assign(clean_dept = df.dept.apply(field_matching.clean_fieldnames))

## Perform matching

In [80]:
all_fields = df.clean_dept.value_counts()
fields_cum = (all_fields/float(all_fields.sum())).cumsum()

In [81]:
fields_cum[:20]

chemistry                 0.022949
physics                   0.041432
psychology                0.054787
mechanical engineering    0.066243
medicine                  0.077611
biology                   0.088313
mathematics               0.098421
education                 0.107444
computer science          0.115709
economics                 0.122769
civil engineering         0.129751
chemical engineering      0.135935
electrical engineering    0.142003
engineering               0.147095
biochemistry              0.151734
biological sciences       0.156107
nursing                   0.160462
history                   0.164741
pharmacy                  0.169002
law                       0.173107
Name: clean_dept, dtype: float64

In [82]:
fields_cum[10000:10010]

computing & informatics                            0.664681
pulmonary & critical care medicine                 0.664687
neuropsicología                                    0.664694
global ecology                                     0.664700
biochemistry, molecular biology, and biophysics    0.664707
history and english                                0.664713
computer science & information systems             0.664720
dept. of physiology anatomy and genetics           0.664727
department structures                              0.664733
european ethnology                                 0.664740
Name: clean_dept, dtype: float64

In [83]:
fields_cum[5000], len(fields_cum)

(0.6195544496270875, 1029974)

In [84]:
field_matching.perform_labeling("soil science"), field_matching.perform_labeling("naval engineering ")

(([('blacklist', 'soil science', 'soil sciences', 'editdist')], ''),
 ([('engineering', 'engineering', 'engineering', 'full_substring')], ' naval'))

In [85]:
black_root_re = re.compile(r"("+r"|".join([x.strip() for x in open("./black-list-roots.txt")])+r")[a-z]+($|\b)")

In [86]:
field_matching.clean_fieldnames('soil science')

'soil science'

In [ ]:
errored = []
errors = 0
matches = []
remains = []
for v, f in tqdm(enumerate(all_fields.items())):
    #print(v,f)
    if v % 100000 == 0:
        print(v, errors)

    field, count = f
    field = field.replace("\t"," ").replace("\n", " ")
    try:
        labels, remaining_string = field_matching.perform_labeling(field)

        for label in labels:
            matches.append((field ,v) + label)

        if black_root_re.search(field):
            matches.append((field ,v, "","","","blacklist"))
        elif not len(labels):
            matches.append((field ,v, "","","","no_match"))

        remains.append((field,str(v),remaining_string, str(count)))
    except UnicodeEncodeError:
        errors += 1
    except KeyboardInterrupt:
        break
    except:
        errored.append(field)

56it [00:00, 544.24it/s]

0 0


100023it [13:54, 125.57it/s]

100000 0


200013it [30:21, 90.69it/s] 

200000 0


209690it [31:58, 92.99it/s] 

In [ ]:
field_matches = pd.DataFrame(matches,
                             columns=['fieldname','index','matched_field','field_term','matched_text','matchtype'])

In [ ]:
field_matches = field_matches.rename(index=str, columns={"index":"field_index"})

In [ ]:
field_matches.to_csv("../../data/all_field_matches.csv",index=False, encoding="utf8")

In [93]:
field_matches.head()

,fieldname,field_index,matched_field,field_term,matched_text,matchtype
0,chemistry,0,chemistry,chemistry,chemistry,exact_full
1,physics,1,physics,physics,physics,exact_full
2,psychology,2,psychology,psychology,psychology,exact_full
3,mechanical engineering,3,engineering,engineering,engineering,full_substring
4,medicine,4,blacklist,medicine,medicine,exact_full


In [97]:
remains[:5]

[('chemistry', '0', '', '136678'),
 ('physics', '1', '', '110080'),
 ('psychology', '2', '', '79540'),
 ('mechanical engineering', '3', ' mechanical', '68230'),
 ('medicine', '4', '', '67707')]

In [99]:
stats = pd.DataFrame(remains, columns=['fieldname','index','term','count'])
stats.to_csv("all_field_remains_and_counts.csv",index=False, encoding="utf8")

In [100]:
df.shape

(7726874, 16)

# Combining matches with data

In [101]:
df.shape

(7726874, 16)

In [102]:
df = df.assign(row_index=range(df.shape[0]))

In [103]:
field_matched_df = pd.merge(df, field_matches,left_on="clean_dept",right_on="fieldname",how="left")

In [104]:
field_matched_df.shape

(9101934, 23)

In [105]:
field_matched_df.to_csv("cleaned_all_affiliations.csv",encoding="utf8")

In [106]:
!gzip cleaned_all_affiliations.csv